In [3]:
import pandas as pd
from utils import aggregate_expert_opinions
metric_results = {}
llm_type = 'gpt'
num_seed = 1
for dataset_name in ['en', 'eu', 'jp']:
    gold = pd.read_csv(f'../step5-close_llm_baseline/data/{dataset_name}.csv')[['job_title', 'job_description', 'task', 'bert_task']]
    gold['bert_task'] = gold['bert_task'].apply(eval)
    for seed in range(num_seed):
        ans = []
        for turn in range(8):
            df = pd.read_csv(f'../step5-close_llm_baseline/results/{dataset_name}/{llm_type}/seed{seed}/result{turn}.csv') 
            ans.append(df)
        ans = pd.concat(ans)
        ans.columns = ['job_title', 'job_description', f'{llm_type}_task_seed{seed}']
        ans[f'{llm_type}_task_seed{seed}'] = ans[f'{llm_type}_task_seed{seed}'].apply(eval)
        gold = pd.merge(gold, ans, on=['job_title', 'job_description'])
    gold['gpt_task'] = gold.apply(lambda x: aggregate_expert_opinions([x[f'{llm_type}_task_seed{i}'] for i in range(num_seed)], top_num=10)[0], axis=1)
    gold = gold[['job_title', 'job_description', 'task', f'{llm_type}_task', 'bert_task']]
    gold.to_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv', index=None)
    gold.to_excel(f'results/{dataset_name}/{dataset_name}-{llm_type}.xlsx', index=None)

In [43]:
for dataset_name in ['eu']:
    gold = pd.read_excel(f'results/{dataset_name}/{dataset_name}-{llm_type}-annotated.xlsx')
    gold['bert_task'] = gold['bert_task'].apply(eval)
    gold['gpt_task'] = gold['gpt_task'].apply(eval)
    gold = gold.fillna(0)
    gold['task'] = gold.apply(lambda x: x['gpt_task'][0] if x['annotated'] == 1 else x['task'], axis=1)
    gold = gold.drop('annotated', axis=1)
    gold.to_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv', index=None)

In [4]:
import pandas as pd
metric_results = {}
llm_type = 'gpt'
for dataset_name in ['en', 'eu', 'jp']:
    metric_results[dataset_name] = {}
    ans = pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv')
    ans[f'{llm_type}_task'] = ans[f'{llm_type}_task'].apply(eval)
    for k in [5, 10]:
        hits = ans.apply(lambda x: x['task'] in x[f'{llm_type}_task'][:k], axis=1).sum() / len(ans)
        metric_results[dataset_name][f"hits@{k}"] = hits
pd.DataFrame(metric_results)

,en,eu,jp
hits@5,0.654135,0.44,0.203883
hits@10,0.684211,0.52,0.349515


In [48]:
import pandas as pd
metric_results = {}
for dataset_name in ['en', 'eu', 'jp']:
    metric_results[dataset_name] = {}
    gold = pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv')
    gold['bert_task'] = gold['bert_task'].apply(eval)
    for k in [5, 10]:
        hits_10 = gold.apply(lambda x: x['task'] in x[f'bert_task'][:k], axis=1).sum() / len(gold)
        metric_results[dataset_name][f"hits@{k}"] = hits_10
pd.DataFrame(metric_results)

,en,eu,jp
hits@5,0.255639,0.16,0.147059
hits@10,0.315789,0.24,0.205882
